In [2]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import dataAck
import curveTreeDB

            

In [ ]:
modelCount, modelSplitByTicker, predictionCount, _unused = curveTreeDB.getModelCounts(params.curveModels)

In [ ]:
modelCount

In [ ]:
modelSplitByTicker

In [ ]:
predictionCount

In [ ]:
modelCount, modelSplitByTicker, predictionCount, numPredictors = curveTreeDB.getModelCounts(params.treeModels)

In [ ]:
modelCount

In [ ]:
modelSplitByTicker

In [ ]:
predictionCount

In [ ]:
numPredictors

In [6]:
sharpeDifference, sharpeDifferenceSlippage = curveTreeDB.getModelPerformance(params.curveModels)

({'AGG': {'<0': 70, '>=0': 7},
  'DIA': {'<0': 34, '>=0': 25},
  'DVY': {'<0': 39, '>=0': 31},
  'DXJ': {'<0': 7, '>=0': 50},
  'EEM': {'<0': 4, '>=0': 52},
  'EFA': {'<0': 7, '>=0': 56},
  'EWC': {'<0': 21, '>=0': 48},
  'EWG': {'<0': 17, '>=0': 33},
  'EWH': {'<0': 18, '>=0': 23},
  'EWJ': {'<0': 1, '>=0': 54},
  'EWT': {'<0': 21, '>=0': 43},
  'EWU': {'<0': 8, '>=0': 52},
  'EWW': {'<0': 27, '>=0': 36},
  'EWY': {'<0': 17, '>=0': 30},
  'EWZ': {'<0': 7, '>=0': 48},
  'FEZ': {'<0': 3, '>=0': 48},
  'FXE': {'<0': 0, '>=0': 131},
  'FXI': {'<0': 0, '>=0': 58},
  'GDX': {'<0': 0, '>=0': 57},
  'GLD': {'<0': 22, '>=0': 40},
  'IAU': {'<0': 7, '>=0': 47},
  'IBB': {'<0': 48, '>=0': 12},
  'IEF': {'<0': 75, '>=0': 8},
  'IJH': {'<0': 43, '>=0': 13},
  'IJR': {'<0': 61, '>=0': 36},
  'ILF': {'<0': 7, '>=0': 55},
  'ITB': {'<0': 35, '>=0': 12},
  'IVE': {'<0': 40, '>=0': 37},
  'IVV': {'<0': 30, '>=0': 23},
  'IVW': {'<0': 51, '>=0': 20},
  'IWB': {'<0': 61, '>=0': 29},
  'IWD': {'<0': 37, '

In [7]:
import json
def getModelProgress():
    toReturn = {}
    
    modelCount, modelSplitByTicker, predictionCount, _unused = curveTreeDB.getModelCounts(params.curveModels)
    sharpeDifference, sharpeDifferenceSlippage = curveTreeDB.getModelPerformance(params.curveModels)
    
    ##FIRST COLUMNS ASSUMED TO BE TICKER
    newColumns = ["Total Count"]
    newRows = []
    for ticker in sorted(modelSplitByTicker):
        newRows.append([ticker, modelSplitByTicker[ticker]])
    
    toReturn["curveSplitByTickerCols"] = json.dumps(newColumns)
    toReturn["curveSplitByTickerRows"] = json.dumps(newRows)
    
    newColumns = ["2", "3", "5", "7", "10", "15"]
    newRows = []
    for ticker in sorted(predictionCount):
        thisRow = []
        for col in newColumns:
            if int(col) not in predictionCount[ticker]:
                thisRow.append(0)
            else:
                thisRow.append(predictionCount[ticker][int(col)])
        
        newRows.append([ticker] + thisRow)
    
    toReturn["curveSplitByPredictionLengthCols"] = json.dumps(newColumns)
    toReturn["curveSplitByPredictionLengthRows"] = json.dumps(newRows)
    
    
    newColumns = ["<0", ">=0"]
    newRows = []
    for ticker in sorted(sharpeDifference):
        thisRow = []
        for col in newColumns:
            if col not in sharpeDifference[ticker]:
                thisRow.append(0)
            else:
                thisRow.append(sharpeDifference[ticker][col])
        
        newRows.append([ticker] + thisRow)
    
    toReturn["curveSharpeDifferenceCols"] = json.dumps(newColumns)
    toReturn["curveSharpeDifferenceRows"] = json.dumps(newRows)
    
    newColumns = ["<0", ">=0"]
    newRows = []
    for ticker in sorted(sharpeDifferenceSlippage):
        thisRow = []
        for col in newColumns:
            if col not in sharpeDifferenceSlippage[ticker]:
                thisRow.append(0)
            else:
                thisRow.append(sharpeDifferenceSlippage[ticker][col])
        
        newRows.append([ticker] + thisRow)
    
    toReturn["curveSharpeDifferenceSlippageCols"] = json.dumps(newColumns)
    toReturn["curveSharpeDifferenceSlippageRows"] = json.dumps(newRows)
    
    
    
    
    modelCount, modelSplitByTicker, predictionCount, numPredictors = curveTreeDB.getModelCounts(params.treeModels)
    sharpeDifference, sharpeDifferenceSlippage = curveTreeDB.getModelPerformance(params.treeModels)
    newColumns = ["Total Count"]
    newRows = []
    for ticker in sorted(modelSplitByTicker):
        newRows.append([ticker, modelSplitByTicker[ticker]])
    
    toReturn["treeSplitByTickerCols"] = json.dumps(newColumns)
    toReturn["treeSplitByTickerRows"] = json.dumps(newRows)
    
    newColumns = ["2", "3", "5", "7", "10", "15"]
    newRows = []
    for ticker in sorted(predictionCount):
        thisRow = []
        for col in newColumns:
            if int(col) not in predictionCount[ticker]:
                thisRow.append(0)
            else:
                thisRow.append(predictionCount[ticker][int(col)])
        
        newRows.append([ticker] + thisRow)
    
    toReturn["treeSplitByPredictionLengthCols"] = json.dumps(newColumns)
    toReturn["treeSplitByPredictionLengthRows"] = json.dumps(newRows)
    
    
    
    newColumns = [str(item) for item in range(30)][2:]
    newRows = []
    for ticker in sorted(numPredictors):
        thisRow = []
        for col in newColumns:
            if int(col) not in numPredictors[ticker]:
                thisRow.append(0)
            else:
                thisRow.append(numPredictors[ticker][int(col)])
        
        newRows.append([ticker] + thisRow)
    
    toReturn["treeNumPredictorsCols"] = json.dumps(newColumns)
    toReturn["treeNumPredictorsRows"] = json.dumps(newRows)
    
    
    newColumns = ["<0", ">=0"]
    newRows = []
    for ticker in sorted(sharpeDifference):
        thisRow = []
        for col in newColumns:
            if col not in sharpeDifference[ticker]:
                thisRow.append(0)
            else:
                thisRow.append(sharpeDifference[ticker][col])
        
        newRows.append([ticker] + thisRow)
    
    toReturn["treeSharpeDifferenceCols"] = json.dumps(newColumns)
    toReturn["treeSharpeDifferenceRows"] = json.dumps(newRows)
    
    newColumns = ["<0", ">=0"]
    newRows = []
    for ticker in sorted(sharpeDifferenceSlippage):
        thisRow = []
        for col in newColumns:
            if col not in sharpeDifferenceSlippage[ticker]:
                thisRow.append(0)
            else:
                thisRow.append(sharpeDifferenceSlippage[ticker][col])
        
        newRows.append([ticker] + thisRow)
    
    toReturn["treeSharpeDifferenceSlippageCols"] = json.dumps(newColumns)
    toReturn["treeSharpeDifferenceSlippageRows"] = json.dumps(newRows)
    return toReturn
   

In [8]:
getModelProgress()

{'curveSharpeDifferenceCols': '["<0", ">=0"]',
 'curveSharpeDifferenceRows': '[["AGG", 70, 7], ["DIA", 34, 25], ["DVY", 39, 31], ["DXJ", 7, 50], ["EEM", 4, 52], ["EFA", 7, 56], ["EWC", 21, 48], ["EWG", 17, 33], ["EWH", 18, 24], ["EWJ", 1, 54], ["EWT", 21, 44], ["EWU", 8, 52], ["EWW", 28, 37], ["EWY", 17, 30], ["EWZ", 7, 48], ["FEZ", 3, 48], ["FXE", 0, 131], ["FXI", 0, 58], ["GDX", 0, 57], ["GLD", 22, 40], ["IAU", 8, 49], ["IBB", 48, 12], ["IEF", 75, 8], ["IJH", 43, 13], ["IJR", 61, 36], ["ILF", 7, 55], ["ITB", 37, 12], ["IVE", 40, 37], ["IVV", 30, 23], ["IVW", 51, 20], ["IWB", 61, 29], ["IWD", 39, 18], ["IWF", 46, 7], ["IWM", 25, 30], ["IWN", 40, 35], ["IWO", 34, 18], ["IWR", 62, 27], ["IYF", 8, 45], ["IYR", 40, 40], ["IYT", 8, 40], ["KBE", 4, 75], ["KRE", 23, 25], ["LQD", 48, 12], ["MDY", 41, 11], ["OEF", 40, 28], ["OIH", 0, 68], ["QQQ", 42, 27], ["RSP", 31, 19], ["SDY", 68, 37], ["SHY", 80, 5], ["SLV", 11, 88], ["SMH", 27, 29], ["SOXX", 9, 44], ["SPY", 26, 28], ["TIP", 53, 15], ["TLT